In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from time import sleep

def get_last_page(url: str) -> str:
    pages = get(url)
    pages_lida = bs(pages.text, 'html.parser')
    links = pages_lida.find('span', {'class': 'wp-pager-pages'}).find_all('a')
    return max([link.get_text() for link in links])

def get_rank(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    rank_tag = ler_pg.find_all('td', {'class':'IndividualStatsRank'})
    ranks = [rank.get_text() for rank in rank_tag]
    return ranks

def get_player(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    player_tag = ler_pg.find_all('a', {'class':'StatsCenterPlayerName'})
    players = [nome.find('span', {'class':'hidden-xs'}).get_text() for nome in player_tag]
    return players

def get_team(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    team_tag = ler_pg.find_all('td', {'class':'StatsCenterTeams'})
    team = []
    for n in team_tag:
        if n.find('span', {'class':'visible-sm-inline'}) != None:
            team.append(n.find('span', {'class':'visible-md-inline'}).get_text())
        else:
            team.append('N/A')
    #team = [n.find('span', {'class':'visible-sm-inline'}).get_text() for n in team_tag]
    return team

def get_games(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    games = [n.get_text()[0:] for n in games_tag[0::2]]
    return games

def get_index(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    index_r = [n.get_text()[0:] for n in games_tag[1::4]]
    return index_r

def get_aver(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    aver = [n.get_text()[0:] for n in games_tag[2::4]]
    return aver

def get_per(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    per_40 = [n.get_text()[0:] for n in games_tag[3::4]]
    return per_40

def get_minutes(url):
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    minutes = [n.get_text()[0:] for n in games_tag[1::2]]
    return minutes

def get_season(url):
    season = ler_pg.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find('option', {'selected':'selected'}).get_text()
    return season

In [2]:
base_url = 'https://www.adidasngt.com/'
stats = f'{base_url}u18/game-center/statistics'

stats_get = get(stats)
stats_page = bs(stats_get.text, 'html.parser')

In [3]:
seasons = stats_page.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find_all('option')

season_dict = {season.get_text() : season.get('value')[11:] for season in seasons}

link_seasons = [f'{stats}?mode=Leaders&entity=Players&seasonmode=Single&seasoncode={season}&cat=TimePlayed&agg=Accumulated' for season in season_dict.values()]

link_seasons

['https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTI20&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTV20&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTB19&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTK19&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTM19&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTV19&cat=TimePlayed&agg=Accumulated',
 'https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=

In [5]:
lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_season = []
lista_minutes = []

for url in link_seasons:
    season_pg = get(url)
    season_page = bs(season_pg.text, 'html.parser')
    if season_page.find('span', {'class': 'wp-pager-pages'}):
        last_page = int(get_last_page(url))
        last_url = ['{}&page={}'.format(url, n) for n in range(1, last_page+1)]
        for urls in last_url:
            pagina = get(urls)
            ler_pg = bs(pagina.text, 'html.parser')
            lista_rank += get_rank(urls)
            lista_player += get_player(urls)
            lista_team += get_team(urls)
            lista_games += get_games(urls)
            lista_minutes += get_minutes(urls)
            print(urls)
            lista_season += [get_season(urls)] * len(get_rank(urls))
        #print(lista_games, lista_minutes)
print('ACABOU!')

https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTV20&cat=TimePlayed&agg=Accumulated&page=1
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTV20&cat=TimePlayed&agg=Accumulated&page=2
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTB19&cat=TimePlayed&agg=Accumulated&page=1
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTB19&cat=TimePlayed&agg=Accumulated&page=2
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTK19&cat=TimePlayed&agg=Accumulated&page=1
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=JTK19&cat=TimePlayed&agg=Accumulated&page=2
https://www.adidasngt.com/u18/game-center/statistics?mode=Leaders&entity=Pla

In [6]:
'''lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []'''

listao = zip(lista_season, lista_rank, lista_player, lista_team, lista_games, lista_minutes)
for l in listao:
    print(l)

('2020-21 VALENCIA', '1', 'KASIAMA, KENNY', 'U18 LDLC ASVEL Villeurbanne', '4', '138:57')
('2020-21 VALENCIA', '2', 'RAYNAUD, MAXIME', 'U18 Nanterre 92', '4', '138:04')
('2020-21 VALENCIA', '3', 'HOUINSOU, KYMANY', 'U18 LDLC ASVEL Villeurbanne', '4', '136:08')
('2020-21 VALENCIA', '4', 'SOCHAN, JEREMY', 'U18 ratiopharm Ulm', '4', '124:02')
('2020-21 VALENCIA', '5', 'LOEMBA, MARC-ANTOINE', 'U18 Nanterre 92', '4', '121:38')
('2020-21 VALENCIA', '6', 'MONTERO, JEAN', 'U18 Herbalife Gran Canaria', '4', '120:50')
('2020-21 VALENCIA', '7', 'ENSMINGER, JACOB', 'U18 ratiopharm Ulm', '4', '117:08')
('2020-21 VALENCIA', '8', 'DONNADIEU, MATEO', 'U18 Nanterre 92', '4', '116:25')
('2020-21 VALENCIA', '9', 'BAGATSKIS, FREDS', 'U18 Valencia Basket', '4', '114:54')
('2020-21 VALENCIA', '10', 'MANET, NOAH', 'U18 LDLC ASVEL Villeurbanne', '4', '114:07')
('2020-21 VALENCIA', '11', 'ALLEN, MIGUEL', 'U18 Joventut Badalona', '4', '112:20')
('2020-21 VALENCIA', '12', 'NGUFOR, DANIEL', 'U18 Nanterre 92', '4'

In [7]:
stats_dict = {'season':lista_season, 'rank':lista_rank, 'player':lista_player, 'team':lista_team, 'games':lista_games, 'minutes':lista_minutes}

df = pd.DataFrame(stats_dict)

df

,season,rank,player,team,games,minutes
0,2020-21 VALENCIA,1,"KASIAMA, KENNY",U18 LDLC ASVEL Villeurbanne,4,138:57
1,2020-21 VALENCIA,2,"RAYNAUD, MAXIME",U18 Nanterre 92,4,138:04
2,2020-21 VALENCIA,3,"HOUINSOU, KYMANY",U18 LDLC ASVEL Villeurbanne,4,136:08
3,2020-21 VALENCIA,4,"SOCHAN, JEREMY",U18 ratiopharm Ulm,4,124:02
4,2020-21 VALENCIA,5,"LOEMBA, MARC-ANTOINE",U18 Nanterre 92,4,121:38
...,...,...,...,...,...,...
5660,2002-03,89,"PSIRIS, SOTIRIOS",U18 AEK ATHENS,2,4:00
5661,2002-03,90,"DRITSOS, GEORGIOS",U18 AEK ATHENS,1,2:00
5662,2002-03,90,"KIRIAKIDIS, SAVAS",U18 AEK ATHENS,1,2:00
5663,2002-03,90,"KORKIDIS, EVANGELOS",U18 AEK ATHENS,1,2:00


In [8]:
df.to_csv('U18_minutes.csv')